In [43]:
from numpy import *

In [151]:
# 数据集有足够的间隔，虽比较分散，但相对容易收敛
def load_simp_data2():
    dataMat = matrix(array([1, 5,2 ,2,3 ,1,4, 6,6 ,8,6 ,5,7 ,9,8, 7,9 ,8,10, 2]).reshape(-1,2))
    classLabels = [1, 1, -1, -1, 1, -1, 1, 1, -1, -1]
    
    return dataMat ,classLabels

h
# 数据集的数据间隔不大，对数据集的划分要足够小，否则不会收敛
def load_simp_data1():
    dataMat = matrix([
             [1., 2.1]
            ,[2., 1.1]
            ,[1.3, 1.]
            ,[1. , 1.]
            ,[2. , 1.]
            
            
            
            
        ])
    classLabels = [1.0, 1.0, -1.0, -1.0, 1.0]
    return dataMat,classLabels

def stumpclassify(dataMatrix,dimen,threshVal,threshIneq):
# 实现弱分类器功能
    '''
    dataMatrix:  输入的数据集
    dimen：      数据集的特征值所在列
    threshVal:   特征值所在列的判断阈值
    threshIneq:  阈值的边界方向 大于/小于等于


    '''

    retArray = ones((shape(dataMatrix)[0], 1))
    if threshIneq == 'lt' :
        retArray[dataMatrix[:, dimen] <= threshVal] = -1.0
    else :
        retArray[dataMatrix[:, dimen] > threshVal ] = -1.0
    return retArray


    

def buildStump(dataArr, classLabels, D):
    '''
    此函数返回该数据集中错误率最少的弱分类器的参数（最佳特征值的列，该列的阈值，阈值的方向（大于还是小于),错误率，当前轮的最佳预测值
    
    dataArr:      输入的数据集
    classLabels:  输入数据集的标签
    D          :  数据集中每个特征列的权重
    
    
    '''
    dataMatrix = mat(dataArr)
    labelMat = mat(classLabels).T
    m, n = shape(dataMatrix)
    # 修改后自动计算步长
    dataMin = dataMatrix.min()
    dataMax = dataMatrix.max()
    numSteps = int( dataMax - dataMin)
    if numSteps <10:
        numSteps = numSteps*m
    else:
        numSteps = numSteps*2
    print('numSteps is :{}'.format(numSteps))
#     numSteps = 10.0
    bestStump = {}
    bestClasEst = mat(zeros((m,1))) 
    minError = inf
    for i in range(n):
        rangeMin = dataMatrix[:,i].min()
        rangeMax = dataMatrix[:,i].max()
        stepSize = (rangeMax - rangeMin) / numSteps
        for j in range(-1,int(numSteps)+1):
            for inequal in ['lt','gt']:
                threshVal = (rangeMin + float(j) * stepSize)
                predictedVals = stumpclassify(dataMatrix, i, threshVal,inequal)
                errArr = mat(ones((m,1)))
                errArr[predictedVals == labelMat] = 0
                weightedError = D.T * errArr
#                 print('split: dim {}, thresh {}, thresh ineqal:{}, the weighted error is {}'.format(i,threshVal,inequal,weightedError))
                if weightedError < minError :
                    minError = weightedError
                    bestClasEst = predictedVals.copy()
                    bestStump['dim'] = i     #特征维度
                    bestStump['thresh'] = threshVal # 特征阈值
                    bestStump['ineq'] = inequal  # 判断标准 大于/小于
                    bestStump['errcount'] = np.count_nonzero(errArr)
#     print('bestStump :{}, minError:{}, bestClasEst :{}'.format(bestStump,minError,bestClasEst))
    return bestStump ,minError, bestClasEst


#### 弱分类器的权重alpha的计算公式:
> 1、弱分类器的阈值选择是有讲究的，阈值的选择要能够分隔相邻的不同类数据。这时对步长设置的要求越高。  
2、数据直接间隔越小，阈值要选的越精细，步长设置要使得阈值达到分隔不同数据的要求。例如，数据集1的数据间隔小，这时步长要设置精细些，反之数据集2的数据间隔大，对步长的要求就不高。  


(1)、$\alpha = \frac{1}{2}ln(\frac{1-\varepsilon}{\varepsilon})$ 
>1、当一个弱分类器的分类正确率为50%时，$\alpha = 0$，所以一般弱分类器的正确率要求高于50%。  
2、公式中的分类错误率$\varepsilon$是分类器权重$\alpha$累加影响下的数值，而不是特征值预测值错误个数比例。  
3、正确率越高的弱分类器$\alpha$系数越大，对分类错误的特征值累加结果的纠偏能力就越强。  
4、adaboost算法，除了第一轮中D是根据样本数量均分的，其余每轮都追求将上一轮错误分类的样本在本轮（对上轮分类错误的样本算法会在本轮增大错误样本点的权重D，算法是求最小错误率，所以上轮的错误样本会在本轮优先进行纠正）进行正确分类，且追求在本轮中分类正确。所以一般都是后面轮错误率$\varepsilon$较小，这样就使得错误率较小的弱分类器，其对应的$\alpha$值较大。  


#### 弱分类器中每个样本点的权重D的计算公式：
利用预测值和原标签值的点积$ y' \cdot y \cdot (-a)$，如果两者相同(正确分类的样本)，结果为$(+1)*(-a)$,如果两者不同(分类错误的样本)，结果为
$(-1) * (-a) = a $

> 1、对正确分类的样本，下一轮分类时样本的权重D值较小；分类不正确的样本，下一轮分类时的样本的权重值较大。  
2、弱分类器在找本轮最佳分类器的参数时（维度，阈值，阈值方向），就是以计算权重后的样本错误率为参考的，越小的错误率越佳；所以，上一轮分类错误的样本点一般会在本轮的算法下得到最大程度的纠正。  
3、


(2.1)、正确分类样本的权重：$D_i^{(t+1)} = \frac{D_i^{(t)} e^{-\alpha}}{Sum(D)}$

(2.2)、未正确分类样本的权重：$D_i^{(t+1)} = \frac{D_i^{(t)} e^{\alpha}}{Sum(D)}$


In [152]:
def adaboostTrainDS(dataArr, classLabels,numIt = 40):
    weakClassArr = []
    m = shape(dataArr)[0]
    # 修改后自动初始化权重
    D = mat(ones((m,1)))/m
    aggClassEst = mat(zeros((m,1)))
    for i in range(numIt) :
        count = i + 1
        print('*'*80)
        print('This is No.{}...'.format(count))
        bestStump, error, classEst = buildStump(dataArr, classLabels,D)
        print('D: {}'.format(D.T))
        print('bestStump :{}, minError:{}, bestpreditVals:{}'.format(bestStump,error,classEst))
#         print('classEst: {}'.format(classEst.T))
        
        alpha = float((1/2)*log((1.0-error)/max(error, 1e-16)))
        bestStump['alpha'] = alpha
        weakClassArr.append(bestStump)
        print('Adjust alpha: {}'.format(alpha))
        expon = multiply(-1*alpha*mat(classLabels).T,classEst)
        D     = multiply(D,exp(expon))
        print('before adjust D: {}, D.sum() = {}'.format(D.T,D.sum()))
        D     = D/D.sum()
        print('D/D.sum() Adjust D: {}, D.sum() ={}'.format(D.T,D.sum()))
        
        aggClassEst += classEst*alpha
        print('No.{} aggclassEst: {}'.format(count,aggClassEst.T))
        aggErrors = multiply(sign(aggClassEst)!= mat(classLabels).T, ones((m,1)) )
        errorRate = aggErrors.sum()/m
        print('No.{} aggclassEst error rate is: {}'.format(count,errorRate))
        if errorRate == 0 :
            break
    return weakClassArr

        
        
        

In [156]:
datamat,labels = load_simp_data1()
classifierArr = adaboostTrainDS(datamat,labels,40)

********************************************************************************
This is No.1...
numSteps is :5
D: [[ 0.2  0.2  0.2  0.2  0.2]]
bestStump :{'dim': 0, 'errcount': 1, 'ineq': 'lt', 'thresh': 1.3999999999999999}, minError:[[ 0.2]], bestpreditVals:[[-1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]]
Adjust alpha: 0.6931471805599453
before adjust D: [[ 0.4  0.1  0.1  0.1  0.1]], D.sum() = 0.7999999999999999
D/D.sum() Adjust D: [[ 0.5    0.125  0.125  0.125  0.125]], D.sum() =1.0000000000000002
No.1 aggclassEst: [[-0.69314718  0.69314718 -0.69314718 -0.69314718  0.69314718]]
No.1 aggclassEst error rate is: 0.2
********************************************************************************
This is No.2...
numSteps is :5
D: [[ 0.5    0.125  0.125  0.125  0.125]]
bestStump :{'dim': 1, 'errcount': 1, 'ineq': 'lt', 'thresh': 1.0}, minError:[[ 0.125]], bestpreditVals:[[ 1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]]
Adjust alpha: 0.9729550745276565
before adjust D: [[ 0.18898224  0.04724556  0.04724556  0.04724

In [ ]:
def adaClassify(dataToClass, classifierArr):
    datamat = mat(dataToClass)
    m = datamat.shape[0]
    aggClassEst = zeros((m,1))
    
    for i in range(len(classifierArr)):
        classEst = stumpclassify(datamat
                                ,classifierArr['dim']
                                ,classifierArr['thresh']
                                ,classifierArr['ineq']
                                
                                )
        aggClassEst += aggClassEst + classEst * classifierArr['alpha']
        print('The No.{} classEst is {} '.format(i+1, aggClassEst))
    
    